# Описание проекта

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

### Описание данных


В таблицах находится информация по 100 000 месторождениям в каждом регионе. У нас имеется следующая информация:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

# 1. Загрузка и подготовка данных

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from scipy import stats as st
from scipy.stats import t
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.utils import shuffle
import statistics as stat

import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore") 
import seaborn as sns 

In [2]:
#откроем файлы с данными:
df_0 = pd.read_csv('/home/andrey/Datasets/geo_data_0.csv')
df_1 = pd.read_csv('/home/andrey/Datasets/geo_data_1.csv')
df_2 = pd.read_csv('/home/andrey/Datasets/geo_data_2.csv')

In [3]:
#изучим данные
df = [df_0,df_1,df_2]

[print(d.info()) for d in df]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

[None, None, None]

Пропусков в данных нет.

In [4]:
#Удалим столбец id во всех датасетах
df0 = df_0.drop('id', axis=1)
df1 = df_1.drop('id', axis=1)
df2 = df_2.drop('id', axis=1)

# 2. Обучение и проверка модели

In [7]:
model = LinearRegression()
#создадим функцию для обучения и проверки модели
def log_reg(df):
    target = df['product']
    features = df.drop('product', axis=1)
    
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    mean = df['product'].mean()
    r2 = r2_score(target_valid, predicted_valid)
    rmse = (mean_squared_error(target_valid, predicted_valid))**0.5
    cvs = stat.mean(cross_val_score(model, features, target, cv=5))
    mae = mean_absolute_error(target_valid, predicted_valid)
    print('Средний запас сырья в регионе:', round(mean, 2))
    print('')
    print('Средняя оценка качества модели:', cvs)
    print('')
    print('rmse:', rmse)
    print('r2', r2)
    print('mae', mae)
    return target_valid, predicted_valid, mean, rmse, r2

In [8]:
print('Регион 0')
print('')
target_valid_0, predicted_valid_0, mean_0, rmse_0, r2_0= log_reg(df0)

Регион 0

Средний запас сырья в регионе: 92.5

Средняя оценка качества модели: 0.2754913072690447

rmse: 37.5794217150813
r2 0.27994321524487786
mae 30.919600777151313


In [9]:
print('Регион 1')
print('')
target_valid_1, predicted_valid_1, mean_1, rmse_1, r2_1= log_reg(df1)

Регион 1

Средний запас сырья в регионе: 68.83

Средняя оценка качества модели: 0.9996243728923551

rmse: 0.893099286775616
r2 0.9996233978805127
mae 0.718766244212475


In [10]:
print('Регион 2')
print('')
target_valid_2, predicted_valid_2, mean_2, rmse_2, r2_2= log_reg(df2)

Регион 2

Средний запас сырья в регионе: 95.0

Средняя оценка качества модели: 0.1987156246205129

rmse: 40.02970873393434
r2 0.20524758386040443
mae 32.792652105481814


Модель в регионе 1 показала лучший результат по метрикам, однако средний запас сырья в регионе меньше чем в других.

# 3. Подготовка к расчёту прибыли

### Сохраните в коде все ключевые значения для расчётов.

In [12]:
#объявим необходимые переменные
dots= 500 #количество точек для иследования
budget = 10**10 #бюджет проекта
one_well_price = 5*10**7 #стоимость одной скважины
wells_count = budget/one_well_price #количество скважин
one_barrel_profit = 4500 #прибыль с одного барреля
th_barrel_profit = one_barrel_profit*1000 #прибыль с 1 тысячи бареллей

<span style="color:green">  Понятные размерности! Ура!! </span>

###  Посчитайте минимальный средний объём сырья в месторождениях региона, достаточный для его разработки. Напишите выводы.

In [13]:
#рассчитаем минимальный средний объём сырья в месторождениях региона, достаточный для его разработки.
#допустим, что этот показатель мы рассчитываем при нулевой операционной прибыле
min_mean_product = (budget/th_barrel_profit)/wells_count
min_mean_product

11.11111111111111

Исходя из полученных данных, можно сделать вывод, что необходимо минимум в среднем 11.11 тысяч баррелей нефти в месторождениях.

<span style="color:purple"> </span>

### Напишите функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели.


In [14]:
#Напишем функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели.
def profit(target_valid, predicted_valid):
    predicted_valid_sort = pd.Series(predicted_valid)
    predicted_valid_sort = predicted_valid_sort.sort_values(ascending=False)
    target_valid = target_valid.reset_index(drop = True)
    top =  target_valid[predicted_valid_sort.index][:200]
    return top.sum()*th_barrel_profit - budget

In [15]:
df0_max_profit = profit(target_valid_0, predicted_valid_0)
print('операционная прибыль в 0 регионе(млрд.):' , df0_max_profit/1000000000)

операционная прибыль в 0 регионе(млрд.): 123.20826043139853


In [16]:
df1_max_profit = profit(target_valid_1, predicted_valid_1)
print('операционная прибыль в 1 регионе(млрд.):', df1_max_profit/1000000000)

операционная прибыль в 1 регионе(млрд.): 114.15086696681512


In [17]:
df2_max_profit = profit(target_valid_2, predicted_valid_2)
print('операционная прибыль в 2 регионе(млрд.):' , df2_max_profit/1000000000)

операционная прибыль в 2 регионе(млрд.): 117.10349963599832


По топ 200 скважин, самая большая прибыль прогнозирыется в 0 регионе (123 млрд.), на втором месте идёт регион 2 (117 млрд.) и на последнем 1 регион с прибылью 114 млрд.

# 4. Расчёт прибыли и рисков 

### Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

### Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков.

In [22]:
state = np.random.RandomState(12345)

#создадим функцию для bootstrap и нахождения средней прибыли, 95% интервала и риска убытков.
def bootstrap(target_valid, predicted_valid):    
    values = []
    for i in range(1000):
        target_valid = target_valid.reset_index(drop = True)
        target_subsample = target_valid.sample(n = dots, replace = True, random_state = state)
        predicted_subsample = predicted_valid[target_subsample.index]
        values.append(profit(target_subsample, predicted_subsample))
    
    values = pd.Series(values)
    mean = values.mean() #среднее
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem()) #дов. интервал
    risk_of_loss = st.percentileofscore(values, 0) #риск убытков
        
    return values, mean, confidence_interval, risk_of_loss


In [23]:
values_0, mean_0, confidence_interval_0, risk_of_loss_0 = bootstrap(target_valid_0, predicted_valid_0)
values_1, mean_1, confidence_interval_1, risk_of_loss_1 = bootstrap(target_valid_1, predicted_valid_1)
values_2, mean_2, confidence_interval_2, risk_of_loss_2 = bootstrap(target_valid_2, predicted_valid_2)

In [24]:
#сравним среднюю прибыль в тыс.
print('Регион 0:', mean_0/1000)
print('Регион 1:', mean_1/1000)
print('Регион 2:', mean_2/1000)

Регион 0: 93961649.8480237
Регион 1: 94611558.1727724
Регион 2: 93929504.75170603


In [18]:
#сравним доверительные интервалы
print('Регион 0:', confidence_interval_0)
print('Регион 1:', confidence_interval_1)
print('Регион 2:', confidence_interval_2)

Регион 0: (93796203151.4797, 94127096544.56769)
Регион 1: (94488289364.8158, 94734826980.729)
Регион 2: (93761642253.29549, 94097367250.1166)


In [19]:
#и риски убытков
print('Регион 0:', risk_of_loss_0)
print('Регион 1:', risk_of_loss_1)
print('Регион 2:', risk_of_loss_2)

Регион 0: 0.0
Регион 1: 0.0
Регион 2: 0.0


Во всех трёх регионах риск убытков составил 0%, самое высокое среднее и лучший доверительный интервал 95% показал регион номер 1. Исходя из этого рекомендуемым регионом к разарботке стоит выбрать 'регион 1'.